In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

# Small text dataset (context)
context = [
    "The sky is blue and the sun is bright.",
    "Cats are independent animals, but they love attention."
]

# Tokenizer (simplistic)
def tokenize(text):
    return text.lower().split()

# Vocabulary and mappings
vocab = list(set(token for sentence in context for token in tokenize(sentence)))
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

# Reinitialize embedding layer
def reinitialize_embedding():
    global model
    model.embedding = nn.Embedding(len(vocab), embed_dim)
    print(f"Embedding layer reinitialized with vocab size: {len(vocab)}")


# Encode and decode functions
def encode(text):
    global vocab, word_to_idx, idx_to_word

    tokens = tokenize(text)
    for token in tokens:
        if token not in word_to_idx:
            # Add new tokens to the vocabulary
            new_idx = len(vocab)
            vocab.append(token)
            word_to_idx[token] = new_idx
            idx_to_word[new_idx] = token

    return torch.tensor([word_to_idx[token] for token in tokens], dtype=torch.long)

def decode(indices):
    return " ".join(idx_to_word[idx.item()] for idx in indices)

# Attention-based QA model
class AttentionQA(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.query_proj = nn.Linear(embed_dim, embed_dim)
        self.key_proj = nn.Linear(embed_dim, embed_dim)
        self.value_proj = nn.Linear(embed_dim, embed_dim)
        self.output_proj = nn.Linear(embed_dim, vocab_size)

    def forward(self, context, question):
        # Embedding
        context_embed = self.embedding(context)
        question_embed = self.embedding(question)

        # Compute query, key, and value
        query = self.query_proj(question_embed.mean(dim=0, keepdim=True))
        keys = self.key_proj(context_embed)
        values = self.value_proj(context_embed)

        # Compute attention scores and weights
        scores = torch.matmul(query, keys.T) / (keys.size(-1) ** 0.5)
        attention_weights = F.softmax(scores, dim=-1)

        # Compute context vector as weighted sum of values
        context_vector = torch.matmul(attention_weights, values).squeeze(0)

        # Output projection
        logits = self.output_proj(context_vector)
        return logits

# Task 1: Customized Text Generation
def generate_custom_text(prompt, temperature, topk, topp):
    prompt_encoded = encode(prompt)
    generated = prompt_encoded.tolist()

    for _ in range(50):  # Generate up to 50 tokens
        logits = model(
            torch.tensor(generated, dtype=torch.long),
            torch.tensor([generated[-1]], dtype=torch.long)
        )
        logits = logits / temperature

        # Top-k sampling
        topk_probs, topk_indices = torch.topk(F.softmax(logits, dim=-1), k=min(topk, logits.size(-1)))
        topk_probs /= topk_probs.sum()

        # Top-p filtering
        sorted_probs, sorted_indices = torch.sort(topk_probs, descending=True)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
        cutoff_index = (cumulative_probs > topp).nonzero(as_tuple=True)[0].min().item()
        valid_indices = sorted_indices[:cutoff_index + 1]

        # Choose next token
        next_token = random.choices(valid_indices.tolist(), weights=sorted_probs[:cutoff_index + 1].tolist())[0]

        # Add token to generated list
        if next_token >= len(vocab):
            raise ValueError(f"Generated token index {next_token} is out of bounds for vocab size {len(vocab)}.")

        generated.append(next_token)

        if idx_to_word[next_token] == "<eos>":  # End of sequence token
            break

    return decode(torch.tensor(generated))


# Task 2: Sentiment Classification with Custom Tokens
def zero_shot_sentiment(review):
    prompt_template = "The sentiment of the review '{review}' is {label}."
    sentiments = ["positive", "negative"]

    scores = []
    for sentiment in sentiments:
        prompt = prompt_template.format(review=review, label=sentiment)
        prompt_encoded = encode(prompt)

        logits = model(torch.tensor(prompt_encoded[:-1]), torch.tensor([prompt_encoded[-1]], dtype=torch.long))
        sentiment_score = F.softmax(logits, dim=-1)[word_to_idx[sentiment]].item()
        scores.append(sentiment_score)

    best_sentiment = sentiments[scores.index(max(scores))]
    return best_sentiment, scores

# Initialize the model with extra space for dynamic updates
embed_dim = 16
vocab_buffer = 50  # Buffer for additional words
model = AttentionQA(len(vocab) + vocab_buffer, embed_dim)

# Add sentiment labels to vocabulary and reinitialize embedding
if "positive" not in word_to_idx or "negative" not in word_to_idx:
    if "positive" not in word_to_idx:
        word_to_idx["positive"] = len(vocab)
        idx_to_word[len(vocab)] = "positive"
        vocab.append("positive")

    if "negative" not in word_to_idx:
        word_to_idx["negative"] = len(vocab)
        idx_to_word[len(vocab)] = "negative"
        vocab.append("negative")

    # Ensure vocabulary size matches embedding
if len(vocab) > model.embedding.num_embeddings:
    reinitialize_embedding()


# Testing Task 1
configs = [(0.7, 10, 0.9), (1.5, 5, 0.95)]  # Added sample configs for temperature, top-k, top-p

print("Task 1: Customized Text Generation")
for temp, topk, topp in configs:
    generated_text = generate_custom_text("The future of AI lies in", temp, topk, topp)
    print(f"Temperature: {temp}, Top-k: {topk}, Top-p: {topp}")
    print(generated_text)

# Testing Task 2
reviews = [
    "This is an outstanding masterpiece!",
    "I wouldn’t recommend watching this movie.",
    "Decent but forgettable performance."
]

print("\nTask 2: Sentiment Classification with Custom Tokens")
for review in reviews:
    sentiment, scores = zero_shot_sentiment(review)
    print(f"Review: {review}")
    print(f"Predicted Sentiment: {sentiment}")
    print(f"Scores: {scores}")


Task 1: Customized Text Generation
Temperature: 0.7, Top-k: 10, Top-p: 0.9
the future of ai lies in cats the they cats blue cats they sun blue blue sun bright. sky blue sun are sun love are they sun sky love sun bright. they cats bright. cats blue cats sun sun the sky blue cats they the are blue the sun bright. the sun they cats cats are
Temperature: 1.5, Top-k: 5, Top-p: 0.95
the future of ai lies in are love sky are love love are sun sun sky are are love love they love love sky sky love sky are sky sky sun sky sun they sun sun sky sun sun sky love sky love love they sky are sky sun are are they sky they love are

Task 2: Sentiment Classification with Custom Tokens
Review: This is an outstanding masterpiece!
Predicted Sentiment: positive
Scores: [0.020004788413643837, 0.012879157438874245]
Review: I wouldn’t recommend watching this movie.
Predicted Sentiment: positive
Scores: [0.019394857808947563, 0.015383508056402206]
Review: Decent but forgettable performance.
Predicted Sentiment: 

<ipython-input-3-a1cf6c6a5841>:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  logits = model(torch.tensor(prompt_encoded[:-1]), torch.tensor([prompt_encoded[-1]], dtype=torch.long))
